<a href="https://colab.research.google.com/github/Raph3103/hw1up/blob/master/Copie_de_Low_Collateral_Blockchain_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low Collateral Blockchain Project


In [189]:
pip install pandas

In [190]:
import pandas as pd
import pickle

Get the stat file: We will start by obtaining the stat file, which contains relevant data for our analysis.

In [191]:
data = pd.read_pickle('/content/HistoryPoolBorrowersStats.pkl')

df = pd.DataFrame.from_dict(data,orient='index')

chemin_fichier_csv = 'newStats.csv'

df.to_csv(chemin_fichier_csv, index=True)


Add columns and modify the existing file: We will enhance the stat file by adding new columns and modifying existing ones to better suit our analysis requirements.


In [192]:
# Read the CSV file into a dataframe
df = pd.read_csv('newStats.csv')

# Remove columns
columns_to_remove = ['datesSupplyCollateral', 'datesLoan','datesReimbursement','datesWithdrawCollateral','timeUTCFirstAnyTransactionAccount','borrowerAgeInYears','totalLoans','NumLoans','MeanLoans','totalTimeloans','MeanTimeLoans','totalSupplyCollateral','NumSupplyCollateral','MeanSupplyCollateral','totalReimbursements','NumReimbursements','MeanReimbursements','NumTransactions']  # Specify the names of the columns to remove
df = df.drop(columns=columns_to_remove)


# Save the modified dataframe to a new CSV file
df.to_csv('newStats.csv', index=False)

This part transfor the dict from date keys to tomestamps keys and also add a new value named ratio for each timestamsp if size debt = 0 then collateralDebtRatio = infinity

In [193]:
import pandas as pd
import json
from datetime import datetime
import time

# Load the csv file
df = pd.read_csv('newStats.csv')


# Define a helper function to convert a date to a timestamp
def date_to_timestamp(date_str):
    dt_object = datetime.strptime(date_str, "%Y-%m-%d")
    timestamp = int(time.mktime(dt_object.timetuple()))
    return timestamp

# Iterate over the Calendar column
for i, cell in enumerate(df['Calendar']):
    if isinstance(cell, str):
        # Replace single quotes with double quotes
        cell = cell.replace("'", '"')
        # Load the dictionary
        calendar_dict = json.loads(cell)
        new_calendar_dict = {}

        # Iterate over the days in the calendar_dict
        for date_str, day_dict in calendar_dict.items():
            # Convert the date to a timestamp and replace it in the dictionary
            timestamp = date_to_timestamp(date_str)

            # Compute the ratio and insert it into the dictionary
            if day_dict['sizeDebtUSD'] != 0:
                ratio = day_dict['sizeCollateralUSD'] / day_dict['sizeDebtUSD']
            else:
                ratio = float('inf')
            day_dict['collateralDebtRatio'] = ratio

            new_calendar_dict[timestamp] = day_dict

        # Replace the row's value in the DataFrame with the updated dictionary
        df.at[i, 'Calendar'] = json.dumps(new_calendar_dict)
    else:
        continue

# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)


adding most frequent time in the day

In [194]:
import pandas as pd
from datetime import datetime

# Define function to convert UNIX timestamp to time of day category
def timestamp_to_category(timestamp):
    # Convert timestamp to datetime
    dt_object = datetime.fromtimestamp(int(float(timestamp)))  # Modify this line
    hour = dt_object.hour

    # Assign time of day category based on hour
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

# Define function to find the most frequent category in a list
def most_frequent_category(lst):
    return max(set(lst), key=lst.count) if lst else None

# Load the csv file
df = pd.read_csv('newStats.csv')

# Define the timestamp columns to examine
timestamp_cols = ['timeStampsSupplyCollateral', 'timeStampsLoans', 'timeStampsReimbursement', 'timeStampsWithdrawCollateral', 'timeStampFirstAnyTransactionAccount']

# Create a new column for the time of day of most transactions
df['MostFrequentTransactionTimeOfDay'] = ""

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    categories = []
    # For each timestamp column, convert the timestamps to categories and add them to the list
    for col in timestamp_cols:
        if pd.isnull(row[col]):
            continue
        timestamps = str(row[col]).split()
        for timestamp in timestamps:
            categories.append(timestamp_to_category(timestamp))
    # Determine the most frequent category and assign it to the new column
    df.at[i, 'MostFrequentTransactionTimeOfDay'] = most_frequent_category(categories)

# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)


count the number of days with high ratio(1 but we can change the value if needed) for each user, using the new value in the calendar colomn

In [195]:
import json

threshold = 2.2

def count_days_with_high_ratio(calendar_str):
    # Check if calendar_str is a string
    if not isinstance(calendar_str, str):
        # Return None (or 0, depending on your needs) if not
        return None  # or return 0

    # Convert string to dictionary
    calendar = json.loads(calendar_str)

    count = 0
    for day, attributes in calendar.items():
        if 'collateralDebtRatio' in attributes and attributes['collateralDebtRatio'] > threshold:
            count += 1
    return count

# Create a new column for the count of days with collateralDebtRatio > 13
df['DaysWithHighRatio'] = df['Calendar'].apply(count_days_with_high_ratio)
# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)

if the user paid x percent in the last y month the val is true

In [196]:
import json
import time

x_percent=0.1

y_month=9
def loan_repaid_x_percent_in_last_y_months(calendar):
    # Define the timestamp for 9 months before the day you stopped computing the file
    six_months_ago = 1668297600 - y_month*30*24*60*60

    # Load the dictionary from the calendar cell
    if isinstance(calendar, str):
        calendar_dict = json.loads(calendar)
    else:
        return False

    # Initialize the total loan and total repayment amounts
    total_loan = 0
    total_repayment = 0

    # Iterate over the days in the calendar_dict
    for timestamp, day_dict in calendar_dict.items():
        # Convert the timestamp to an integer
        timestamp = int(timestamp)

        # If the timestamp is within the last 6 months and not before the user's first day
        if six_months_ago <= timestamp <= 1668297600:
            # Add the loan and repayment amounts for the day to the totals
            total_loan += day_dict['sizeDebtUSD']
            total_repayment += day_dict['amountPaymentsOnDay']

    # If the total loan amount is 0, return False
    if total_loan == 0:
        return False

    # Calculate the repayment percentage
    repayment_percentage = total_repayment / total_loan

    # If the repayment percentage is at least 66%, return True
    if repayment_percentage >= x_percent:
        return True
    else:
        return False

# Apply the function to the Calendar column to create a new column
df['LoanRepaidxPercentInLastyMonths'] = df['Calendar'].apply(loan_repaid_x_percent_in_last_y_months)


# Save the updated DataFrame back to csv
df.to_csv('newStats.csv', index=False)

Create a function to evaluate user behavior: We will develop a function that determines whether a user is classified as good or bad based on their activities on our platform. This function will help us assess user behavior and identify potential risks.


In [197]:
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

six_months_ago = today - timedelta(days=15*30)  # Assuming each month has 30 days

# Convert the dates to timestamps
timestamp_today = int(today.timestamp())
timestamp_six_months_ago = int(six_months_ago.timestamp())


def evaluate_user_behavior(row):
    conditions_met = 0


    if row['DaysWithHighRatio'] > 60:
        conditions_met += 2
        if row['LoanRepaidxPercentInLastyMonths']:
          conditions_met += 4
    if row['timeStampFirstAnyTransactionAccount'] < timestamp_six_months_ago :
        conditions_met += 1

    if conditions_met==5 or conditions_met == 6 or conditions_met==7 or conditions_met == 3:
        return 'Good'
    elif conditions_met ==0 :
        return 'Bad'
    else:
        return 'Unknown'


Run the function for all rows: We will iterate over each row of the stat file and apply the user evaluation function to determine the classification for each user.


In [198]:
df['user_class'] = df.apply(evaluate_user_behavior, axis=1)

Add the new column to the CSV: After evaluating all the users, we will add the newly generated classification column to the CSV file.


In [199]:
import pandas as pd

df = df.drop(columns='Calendar')

# Save the DataFrame as a CSV file
df.to_csv('updated_data.csv', index=False)

Run a machine learning model: With the updated CSV file, we will utilize a machine learning model to gain insights and predictions based on the user behavior data.


In [200]:
def user_class_distribution(data: pd.DataFrame):
    label_counts = data["user_class"].value_counts().reset_index()
    label_counts.columns = ['Rating', 'Count']
    label_counts['Rating'] = label_counts['Rating'].map({0: 'Bad', 1: 'Good', 2: 'Unknown'})
    fig = px.bar(
        label_counts,
        x='Rating',
        y='Count',
        title='Label Distribution',
        labels={'Rating': 'user_class', 'Count': 'Count'},
        color='Rating',  # Apply colors to each category
        color_discrete_map={'Bad': 'blue', 'Good': 'green', 'Unknown': 'orange',},  # Specify colors
    )

    # Customize layout and appearance
    fig.update_layout(
        xaxis_title="user_class",
        yaxis_title="Count",
        legend_title="Label Category",
        font=dict(size=12),
    )

    fig.show()

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler



# Read the data
df = pd.read_csv('updated_data.csv')
#print(df.describe())
# Drop unnecessary column
df = df.drop(columns=['Unnamed: 0'])
#df = df.drop(columns=['Calendar'])
# Handle timestamp strings
# Handle timestamp strings
for col in ['timeStampsSupplyCollateral', 'timeStampsLoans', 'timeStampsReimbursement', 'timeStampsWithdrawCollateral','maximumDebt', 'atTimeMaximumDebtCollateralProvided']:
    df= df.drop(columns=col)

rows_with_nan = df[df.isna().any(axis=1)]

df.dropna()
# Convert boolean to int
df['LoanRepaidxPercentInLastyMonths'] = df['LoanRepaidxPercentInLastyMonths'].astype(int)

# Handle categorical columns
le = preprocessing.LabelEncoder()
df['MostFrequentTransactionTimeOfDay'] = le.fit_transform(df['MostFrequentTransactionTimeOfDay'])
df['user_class'] = le.fit_transform(df['user_class'])

# Extract features from Calendar dictionary (you need to specify this according to the dictionary's structure)

#print(df.describe())

# Normalize numerical columns
scaler = StandardScaler()
num_cols = ['sizeLoansUSD', 'sizeCollateralUSD', 'sizeReimbursementsUSD', 'numLoansUser', 'ratioCollateralToLoans', 'averageOfDailyCollateralToDebt', 'numTransactionsUser', 'DaysWithHighRatio']
df[num_cols] = scaler.fit_transform(df[num_cols])

# Define features and target
for cols in ['DaysWithHighRatio','timeStampFirstAnyTransactionAccount','LoanRepaidxPercentInLastyMonths']:
  df = df.drop(columns=cols)
X = df.drop('user_class', axis=1)
y = df['user_class']

#print(X.head())
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Drop rows with NaN values
X_train = X_train.dropna()
y_train = y_train[X_train.index]

In [202]:

# Create the KNN model
knn = KNeighborsClassifier(n_neighbors=3)

# Train the model
knn.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8125


In [203]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Définir les paramètres à tester
param_grid = {
    'n_neighbors': [3],  # Toujours 3 voisins
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2],  # 1 pour la distance de Manhattan, 2 pour la distance euclidienne
}

# Initialiser le modèle k-NN
knn = KNeighborsClassifier()

# Initialiser la recherche sur grille
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

# Effectuer la recherche sur grille sur l'ensemble d'entraînement
grid_search.fit(X_train, y_train)

# Meilleurs paramètres trouvés
best_params = grid_search.best_params_
# Modèle avec les meilleurs paramètres
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test avec le modèle optimisé
predictions = best_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f"Meilleurs paramètres : {best_params}")
print(f'Accuracy avec meilleurs paramètres: {accuracy}')
print('Classification Report avec meilleurs paramètres:\n', report)


Meilleurs paramètres : {'algorithm': 'auto', 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
Accuracy avec meilleurs paramètres: 0.8125
Classification Report avec meilleurs paramètres:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.89      0.91      0.90        43

    accuracy                           0.81        48
   macro avg       0.30      0.30      0.30        48
weighted avg       0.79      0.81      0.80        48



In [204]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import numpy as np

# Create a model
rfc = RandomForestClassifier( n_estimators=100, random_state=42)

# Fit the model
rfc.fit(X_train, y_train)

# Make predictions
y_pred = rfc.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.00      0.00      0.00         3
           2       0.91      0.95      0.93        43

    accuracy                           0.88        48
   macro avg       0.47      0.48      0.48        48
weighted avg       0.84      0.88      0.86        48



Analyze performance: Finally, we will analyze the performance of our machine learning model and evaluate its effectiveness in predicting user behavior accurately.

In [205]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Définir les paramètres à tester
param_grid = {
    'n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
    'max_depth': [None, 10, 20, 30],  # Profondeur maximale de chaque arbre
    'min_samples_split': [2, 5, 10],  # Nombre minimal d'échantillons requis pour diviser un nœud
    'min_samples_leaf': [1, 2, 4],  # Nombre minimal d'échantillons requis à chaque feuille
    'max_features': ['auto', 'sqrt', 'log2'],  # Nombre maximal de fonctionnalités à considérer pour la division d'un nœud
}

# Initialiser le modèle Random Forest
rf = RandomForestClassifier()

# Initialiser la recherche sur grille
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

# Effectuer la recherche sur grille sur l'ensemble d'entraînement
grid_search.fit(X_train, y_train)
# Meilleurs paramètres trouvés
best_params = grid_search.best_params_

# Modèle avec les meilleurs paramètres
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test avec le modèle optimisé
predictions = best_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f"Meilleurs paramètres : {best_params}")
print(f'Accuracy avec meilleurs paramètres: {accuracy}')
print('Classification Report avec meilleurs paramètres:\n', report)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifi

Meilleurs paramètres : {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy avec meilleurs paramètres: 0.875
Classification Report avec meilleurs paramètres:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.89      0.98      0.93        43

    accuracy                           0.88        48
   macro avg       0.30      0.33      0.31        48
weighted avg       0.80      0.88      0.84        48



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-de

In [211]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report



# Initialiser le modèle linéaire (Régression logistique dans ce cas)
model = LogisticRegression()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.8958333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.90      1.00      0.95        43

    accuracy                           0.90        48
   macro avg       0.30      0.33      0.32        48
weighted avg       0.80      0.90      0.85        48



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [212]:
# Importer les bibliothèques nécessaires
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Définir les paramètres à tester
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse de la force de régularisation
    'penalty': ['l1', 'l2'],  # Type de régularisation (L1 ou L2)
}

# Initialiser le modèle de régression logistique
logistic_regression = LogisticRegression(max_iter=1000)

# Initialiser la recherche sur grille
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')

# Effectuer la recherche sur grille sur l'ensemble d'entraînement
grid_search.fit(X_train, y_train)

# Meilleurs paramètres trouvés
best_params = grid_search.best_params_

# Modèle avec les meilleurs paramètres
best_model = grid_search.best_estimator_
# Faire des prédictions sur l'ensemble de test avec le modèle optimisé
predictions = best_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f"Meilleurs paramètres : {best_params}")
print(f'Accuracy avec meilleurs paramètres: {accuracy}')
print('Classification Report avec meilleurs paramètres:\n', report)

Meilleurs paramètres : {'C': 0.001, 'penalty': 'l2'}
Accuracy avec meilleurs paramètres: 0.8958333333333334
Classification Report avec meilleurs paramètres:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.90      1.00      0.95        43

    accuracy                           0.90        48
   macro avg       0.30      0.33      0.32        48
weighted avg       0.80      0.90      0.85        48



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning:


30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solv

In [207]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Initialiser le modèle SVM
model = SVC(kernel='linear', C=1.0, decision_function_shape='ovr')  # 'ovr' pour classification multi-classe

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.8958333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.90      1.00      0.95        43

    accuracy                           0.90        48
   macro avg       0.30      0.33      0.32        48
weighted avg       0.80      0.90      0.85        48



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [213]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Définir les paramètres à tester
param_grid = {
    'C': [0.1, 1, 10],  # Paramètre de régularisation
    'kernel': ['linear', 'rbf', 'poly'],  # Type de noyau
    'degree': [2, 3, 4],  # Degré du noyau polynomial (si applicable)
}

# Initialiser le modèle SVM
svm = SVC()

# Initialiser la recherche sur grille
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')

# Effectuer la recherche sur grille sur l'ensemble d'entraînement
grid_search.fit(X_train, y_train)

# Meilleurs paramètres trouvés
best_params = grid_search.best_params_
# Modèle avec les meilleurs paramètres
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test avec le modèle optimisé
predictions = best_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f"Meilleurs paramètres : {best_params}")
print(f'Accuracy avec meilleurs paramètres: {accuracy}')
print('Classification Report avec meilleurs paramètres:\n', report)

Meilleurs paramètres : {'C': 0.1, 'degree': 2, 'kernel': 'linear'}
Accuracy avec meilleurs paramètres: 0.8958333333333334
Classification Report avec meilleurs paramètres:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.90      1.00      0.95        43

    accuracy                           0.90        48
   macro avg       0.30      0.33      0.32        48
weighted avg       0.80      0.90      0.85        48



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [208]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialiser le modèle XGBoost pour la classification multi-classe
model = XGBClassifier(objective='multi:softmax', num_class=3, seed=42)

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)


Accuracy: 0.8541666666666666
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.25      0.33      0.29         3
           2       0.93      0.91      0.92        43

    accuracy                           0.85        48
   macro avg       0.56      0.58      0.57        48
weighted avg       0.87      0.85      0.86        48



In [209]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Définir les paramètres à tester
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],  # Taux d'apprentissage
    'n_estimators': [50, 100, 200],  # Nombre d'arbres
    'max_depth': [3, 5, 7],  # Profondeur maximale de chaque arbre
    'min_child_weight': [1, 3, 5],  # Poids minimal des enfants (échantillons) nécessaires pour créer un nouveau nœud
    'subsample': [0.8, 1.0],  # Proportion d'échantillons utilisée pour ajuster les modèles
    'colsample_bytree': [0.8, 1.0],  # Proportion de fonctionnalités utilisée pour ajuster les modèles
}

# Initialiser le modèle XGBoost
xgb = XGBClassifier(objective='multi:softmax', num_class=3, seed=42)

# Initialiser la recherche sur grille
grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='accuracy')

# Effectuer la recherche sur grille sur l'ensemble d'entraînement
grid_search.fit(X_train, y_train)
# Meilleurs paramètres trouvés
best_params = grid_search.best_params_

# Modèle avec les meilleurs paramètres
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test avec le modèle optimisé
predictions = best_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Afficher les résultats
print(f"Meilleurs paramètres : {best_params}")
print(f'Accuracy avec meilleurs paramètres: {accuracy}')
print('Classification Report avec meilleurs paramètres:\n', report)

Meilleurs paramètres : {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1.0}
Accuracy avec meilleurs paramètres: 0.875
Classification Report avec meilleurs paramètres:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         3
           2       0.89      0.98      0.93        43

    accuracy                           0.88        48
   macro avg       0.30      0.33      0.31        48
weighted avg       0.80      0.88      0.84        48



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [214]:
# Importer les bibliothèques nécessaires
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Définir les modèles avec les meilleurs paramètres que vous avez trouvés
knn_model = KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='auto', p=2)
rf_model = RandomForestClassifier(max_depth=None, max_features='auto', min_samples_leaf=2, min_samples_split=5, n_estimators=100)
logistic_model = LogisticRegression(C=0.001, penalty='l2')
svm_model = SVC(C=0.1, degree=2, kernel='linear')
xgb_model = XGBClassifier(colsample_bytree=1, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=100, subsample=1)

# Liste des modèles
models = [('knn', knn_model), ('random_forest', rf_model), ('logistic_regression', logistic_model),
          ('svm', svm_model), ('xgboost', xgb_model)]

# Initialiser des listes pour stocker les résultats
accuracy_scores = []
classification_reports = []
confusion_matrices = []

# Entraîner et évaluer chaque modèle
for model_name, model in models:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculer et stocker l'accuracy
    accuracy = accuracy_score(y_test, predictions)
    accuracy_scores.append((model_name, accuracy))

    # Stocker le rapport de classification
    report = classification_report(y_test, predictions, output_dict=True)
    classification_reports.append((model_name, report))

    # Stocker la matrice de confusion
    conf_matrix = confusion_matrix(y_test, predictions)
    confusion_matrices.append((model_name, conf_matrix))
# Afficher les résultats d'accuracy
for model_name, accuracy in accuracy_scores:
    print(f'{model_name} Accuracy: {accuracy}')

# Comparer les modèles avec une visualisation
plt.figure(figsize=(10, 6))
for model_name, conf_matrix in confusion_matrices:
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, label=model_name)
plt.title('Matrices de Confusion Comparatives')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.legend()
plt.show()

# Comparer les rapports de classification
for model_name, report in classification_reports:
    print(f'\n{model_name} Classification Report:\n', report)

# Vous pouvez également comparer d'autres métriques d'évaluation si nécessaire

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning:

`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-de

knn Accuracy: 0.8125
random_forest Accuracy: 0.8958333333333334
logistic_regression Accuracy: 0.8958333333333334
svm Accuracy: 0.8958333333333334
xgboost Accuracy: 0.875


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` p

NameError: ignored

<Figure size 1000x600 with 0 Axes>